In [1]:
!nvidia-smi

Thu Oct  9 13:35:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In this notebook, i'll leverage with PEFT library from HF ecosystem, as well as QLoRA for more memory efficient finetuning

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.2 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login
login()

In [4]:
from datasets import load_dataset
dataset_name = "nisaar/Articles_Constitution_3300_Instruction_Set"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)2_14_15_19_21_Instructionset_train.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3311 [00:00<?, ? examples/s]

In [5]:
dataset[1]

{'instruction': 'Identify and summarize the key legal issues in the provided case.',
 'input': 'Case Citation: Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278. The case revolves around the termination of employment of the respondents by the appellant without assigning any reason by invoking Rule 9(i) of the service rules. The respondents challenged the termination orders and the validity of Rule 9(i) in the High Court under Article 226. The Division Bench of the High Court struck down Rule 9(i) as arbitrary and violative of Article 14. The appellant filed appeals in the Supreme Court against the High Court judgment.',
 'output': "The key legal issues in the case Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. are as follows: 1. The first issue is whether a government company falls under the definition of 'State' as per Article 12 of the Indian Constitution. This is important as the constitutional

In [63]:
from datasets import Dataset

# Apply the formatting function to create the 'text' column
dataset = dataset.map(formatting_func, remove_columns=['instruction', 'input', 'output', 'prompt'])

# Tokenize the dataset
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=max_seq_length
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3311 [00:00<?, ? examples/s]

TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'str'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [7]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'text'],
    num_rows: 3311
})

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype= torch.float,
    bnb_4bit_quant_type="nf4")

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, trust_remote_code = True)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [14]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64


In [24]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [27]:
from transformers import TrainingArguments

In [54]:
output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 1
num_train_epochs = 4
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 20
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [64]:
from trl import SFTConfig

training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    completion_only_loss=False,
)

In [30]:
from trl import SFTTrainer

In [33]:
max_seq_length = 2048

In [43]:
# example: tokenize dataset first
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=max_seq_length
    )

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/3311 [00:00<?, ? examples/s]

In [67]:
def formatting_func(example):
    text = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return {"text": text}

In [65]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_arguments,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:279: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [40]:
from inspect import signature
from trl import SFTTrainer

print(signature(SFTTrainer.__init__))


(self, model: Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel], args: Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType] = None, data_collator: Optional[Callable[[list[Any]], dict[str, Any]]] = None, train_dataset: Union[datasets.arrow_dataset.Dataset, datasets.iterable_dataset.IterableDataset, NoneType] = None, eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None, processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.processing_utils.ProcessorMixin, NoneType] = None, compute_loss_func: Optional[Callable] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], dict]] = None, callbacks: Optional[list[transformers.trainer_callback.TrainerCallback]] = None, optimizers: tuple[typing.Optional[torch.optim.optimizer.Optimizer], typing.Optional[torch.optim.lr_scheduler.LambdaLR

In [59]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)


In [89]:
from datasets import Dataset

# Tokenize the dataset using the 'prompt' column
def tokenize(example):
    tokenized_inputs = tokenizer(
        example["prompt"],
        truncation=True,
        max_length=max_seq_length
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy() # Add labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=['instruction', 'input', 'output', 'prompt']) # Remove original columns

Map:   0%|          | 0/3311 [00:00<?, ? examples/s]

In [72]:
from datasets import load_dataset
dataset_name = "nisaar/Articles_Constitution_3300_Instruction_Set"
dataset = load_dataset(dataset_name, split="train")

In [91]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Define a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Define TrainingArguments (reusing parameters from SFTConfig)
training_args = TrainingArguments(
    output_dir=training_arguments.output_dir,
    per_device_train_batch_size=training_arguments.per_device_train_batch_size,
    gradient_accumulation_steps=training_arguments.gradient_accumulation_steps,
    optim=training_arguments.optim,
    num_train_epochs=training_arguments.num_train_epochs,
    save_steps=training_arguments.save_steps,
    logging_steps=training_arguments.logging_steps,
    learning_rate=training_arguments.learning_rate,
    fp16=training_arguments.fp16,
    max_grad_norm=training_arguments.max_grad_norm,
    max_steps=training_arguments.max_steps,
    warmup_ratio=training_arguments.warmup_ratio,
    lr_scheduler_type=training_arguments.lr_scheduler_type,
    remove_unused_columns=False, # Keep all columns
    # group_by_length is not directly available in TrainingArguments, will skip for now
    # completion_only_loss is specific to SFTTrainer, will skip
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-521743487.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [76]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:279: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Since we encountered persistent issues with the `SFTTrainer`, we will switch to using the standard Hugging Face `Trainer` for finetuning.

In [74]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_arguments,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:279: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/3311 [00:00<?, ? examples/s]

In [82]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: paudelsamir (paudelsamir-hugging-face) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [92]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.559100
2,2.467300
3,2.436600
4,2.016700
5,1.962500
6,2.239700
7,2.370000
8,1.983200
9,2.051000
10,2.067000


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

TrainOutput(global_step=20, training_loss=1.9885198831558228, metrics={'train_runtime': 184.1853, 'train_samples_per_second': 0.217, 'train_steps_per_second': 0.109, 'total_flos': 248396150587392.0, 'train_loss': 1.9885198831558228, 'epoch': 0.012080942313500454})

In [94]:
dataset[0]

{'instruction': 'Analyze and explain the legal reasoning behind the judgment in the given case.',
 'input': 'Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278',
 'output': "The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reason, conferred unfettered power to termin

In [98]:
dataset['output'][0]

"The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reason, conferred unfettered power to terminate employment without hearing. This was deemed arbitrary and violative of principles of natural justice and right to equality under Article 14. Furthermore, the court held that the right to life and livelihood under Article 21 is a


We installed the necessary libraries (like transformers, trl, peft, etc.) and logged into Hugging Face and Weights & Biases.
Data: We loaded a dataset containing legal articles and instructions. We then prepared this data so the model could understand it, which involved formatting the text and converting it into numbers (tokenization) that the model can process.We loaded a large language model (Llama-2) that was already partially trained. To make it more memory-efficient and suitable for fine-tuning on a smaller dataset, we used techniques called Quantization (specifically 4-bit quantization) and PEFT (Parameter-Efficient Fine-Tuning) with LoRA. We used the prepared data to fine-tune the model using the Hugging Face Trainer. This process involved showing the model the examples from the dataset so it could learn to follow the instructions and generate relevant legal reasoning. We encountered some challenges with the SFTTrainer and switched to the standard Trainer to successfully train the model. Finally, we tested the fine-tuned model by giving it a prompt and having it generate a response.